# Tarea Dask

## Instrucciones generales
- Esta tarea debe realizarse de manera individual
- Este notebook (resuelto) debe ser subido al github del proyecto en la carpeta de tareas (creen una carpeta dentro de esa carpeta y agreguen su notebook reuelto)
- Fecha límite: Lunes 25 de noviembre de 2024 a las 11:59 p.m
- Deben realizar las cuatro secciones
- Puedes agregar tantas celdas de código y explicaciones como veas necesario, solo manten la estructura general

Librerias necesarias 

In [3]:

import numpy as np
import pandas as pd
import time

import dask
from dask import delayed, visualize
from dask.distributed import Client, wait
import dask.dataframe as dd
import dask.array as da


## Sección 0 Creación y Configuración del cliente de Dask
Ejercicio 0: Configuración del cliente
1. Crea un cliente local de Dask que inicie un clúster en tu máquina.
2. Configura el cliente para que tenga las siguientes características (elige un par de las opciones de trabajadores e hilos):
    - Número de trabajadores: 2 / 4
    - Memoria máxima por trabajador: 1GB
    - Threads por trabajador: 4 / 2
3. Verifica que el cliente esté funcionando correctamente mostrando:
    - Resumen de los trabajadores activos.
    - Dashboard disponible (URL del panel de control de Dask).
    * Tip: Checa los parámetros del cliente que creeaste.

*Nota*: Puedes hacer que corra en el puerto que desees.

In [ ]:
cluster = LocalCluster(n_workers=2, threads_per_worker=4, memory_limit="1GB", dashboard_address=":8787" )

client = Client(cluster)
print(client)

print("\nResumen de trabajadores:")
print(client.scheduler_info()["workers"])

print(f"\nDashboard disponible en: {client.dashboard_link}")


In [4]:
client = Client(n_workers=4, threads_per_worker=2, memory_limit='1GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 3.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56484,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 3.73 GiB
Comm: tcp://127.0.0.1:56510,Total threads: 2
Dashboard: http://127.0.0.1:56511/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:56487,


## Sección 1 Delayed
Ejercicio 1: Procesamiento de datos 

1. Genera datos simulados (por ejemplo, ventas diarias) para 10 sucursales durante 365 días.
    - Cada sucursal debe tener datos generados aleatoriamente para "Ingresos" y "Costos".
    - Utiliza una función para generar los datos simulados.
2. Usa Dask Delayed para calcular:
    - Las ganancias diarias por sucursal.
    - La sucursal con mayor ganancia promedio.
3. Genera un grafo de tareas que visualice estas operaciones y explica por qué elegiste paralelizar de esa forma, genera una visualización del grafo.

In [5]:
# Paso 1: Generar datos simulados
def generar_datos(sucursal, dias=365):
    np.random.seed(sucursal)  # Asegurar reproducibilidad
    ingresos = np.random.randint(1000, 10000, size=dias)
    costos = np.random.randint(500, 8000, size=dias)
    return pd.DataFrame({"Sucursal": sucursal, "Día": range(1, dias + 1), "Ingresos": ingresos, "Costos": costos})

# Generar datos para 10 sucursales
sucursales = [generar_datos(i) for i in range(1, 11)]

# Paso 2: Definir cálculos con Dask Delayed
@delayed
def calcular_ganancias(df):
    df["Ganancia"] = df["Ingresos"] - df["Costos"]
    return df

@delayed
def promedio_ganancia(df):
    return df["Ganancia"].mean()

@delayed
def encontrar_sucursal_mayor_promedio(promedios):
    return max(promedios, key=lambda x: x[1])

# Aplicar los cálculos con Dask Delayed
ganancias = [calcular_ganancias(df) for df in sucursales]
promedios = [promedio_ganancia(df) for df in ganancias]
mayor_promedio = encontrar_sucursal_mayor_promedio(
    delayed(list)(zip(range(1, 11), promedios))
)

# Paso 3: Generar el grafo de tareas
visualize(mayor_promedio, filename="grafo_tareas", format="png")

# Computar los resultados
result = mayor_promedio.compute()
print(f"Sucursal con mayor ganancia promedio: {result[0]} con promedio {result[1]:.2f}")


ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

## Sección 2 Dask Dataframes
Ejercicio 2: Limpieza y análisis de datos reales

1. Descarga un conjunto de datos masivo (puedes usar la colección de *nycflights* que se encuentra en `data/nycflights/`).
2. Carga los datos en un Dask DataFrame. 
    - Elige adecuadamente el número de particiones (que quepan en memoria de los `workers`)
3. Realiza las siguientes tareas:
    - Limpia los valores faltantes en las columnas `ArrDelay` y `DepDelay`, rellenándolos con la mediana de cada columna.
    - Calcula el retraso promedio (`DepDelay`) por mes y aerolínea.
    - Encuentra el aeropuerto de origen con más vuelos retrasados.

*Nota*: **Evita** convertir el DataFrame a pandas e **intenta** realizar `.compute()` solo cuando sea necesario.

In [6]:
import dask.dataframe as dd

# Paso 1: Cargar el conjunto de datos
ruta_datos = "data/nycflights/*.csv" 
df = dd.read_csv(ruta_datos, assume_missing=True)

print(df.columns)

# Paso 2: Limpiar valores faltantes en ArrDelay y DepDelay
# Calcular la mediana de cada columna
arr_delay_median = df["ArrDelay"].median().compute()
dep_delay_median = df["DepDelay"].median().compute()

# Rellenar valores faltantes con la mediana correspondiente
df["ArrDelay"] = df["ArrDelay"].fillna(arr_delay_median)
df["DepDelay"] = df["DepDelay"].fillna(dep_delay_median)

# Paso 3: Calcular retraso promedio por mes y aerolínea
# Agrupar por mes y aerolínea, luego calcular el promedio
retraso_promedio = df.groupby(["Month", "UniqueCarrier"])["DepDelay"].mean()

# Paso 4: Encontrar el aeropuerto con más vuelos retrasados
# Filtrar vuelos retrasados (retraso > 0)
vuelos_retrasados = df[df["DepDelay"] > 0]

# Contar retrasos por aeropuerto de origen
retrasos_por_aeropuerto = vuelos_retrasados.groupby("Origin")["FlightNum"].count().compute()

# Ordenar y encontrar el aeropuerto con más retrasos
aeropuerto_con_mas_retrasos = retrasos_por_aeropuerto.idxmax()
max_retrasos = retrasos_por_aeropuerto.max()

# Mostrar resultados
print(f"\nRetraso promedio por mes y aerolínea:\n{retraso_promedio.compute()}")
print(f"\nAeropuerto con más vuelos retrasados: {aeropuerto_con_mas_retrasos} ({max_retrasos} retrasos)")


OSError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: data/nycflights/*.csv resolved to no files

## Sección 3 Dask Arrays

Ejercicio 3: Procesamiento numérico avanzado

1. Crea un arreglo de 10,000 x 10,000 con valores aleatorios usando Dask Array, utiliza un tamaño de chunks adecuado, ¿es mejor que sean cuadrados?.
2. Realiza las siguientes operaciones:
    - Calcula la suma de cada fila.
    - Encuentra la fila con el valor máximo promedio.
    - Multiplica todo el arreglo por un factor escalar (por ejemplo, 2.5).
3. Divide el arreglo nuevamente en 100 bloques y compara la rapidez.

In [ ]:

# Crear un arreglo 10,000 x 10,000 con valores aleatorios
# Elegimos chunks cuadrados inicialmente
arreglo = da.random.random((10_000, 10_000), chunks=(1_000, 1_000))
suma_filas = arreglo.sum(axis=1)
promedios_filas = arreglo.mean(axis=1)
fila_max_promedio = promedios_filas.argmax()
arreglo_escalar = arreglo * 2.5
start_time = time.time()
suma_filas_result = suma_filas.compute()
fila_max_promedio_result = fila_max_promedio.compute()
arreglo_escalar_result = arreglo_escalar.compute()
print(f"\nTiempo con chunks de 1,000x1,000: {time.time() - start_time:.2f} segundos")

# Redividir el arreglo en 100 bloques (chunks más pequeños)
arreglo_redividido = arreglo.rechunk((100, 100))

# Repetir las operaciones con los nuevos chunks
start_time = time.time()
suma_filas_result_redividido = arreglo_redividido.sum(axis=1).compute()
fila_max_promedio_result_redividido = arreglo_redividido.mean(axis=1).argmax().compute()
arreglo_escalar_result_redividido = (arreglo_redividido * 2.5).compute()
print(f"Tiempo con chunks de 100x100: {time.time() - start_time:.2f} segundos")

print(f"\nResultados:")
print(f"- Fila con el máximo promedio (chunks originales): {fila_max_promedio_result}")
print(f"- Fila con el máximo promedio (chunks redivididos): {fila_max_promedio_result_redividido}")



Tiempo con chunks de 1,000x1,000: 4.06 segundos
Tiempo con chunks de 100x100: 26.74 segundos

Resultados:
- Fila con el máximo promedio (chunks originales): 3026
- Fila con el máximo promedio (chunks redivididos): 3026


## Sección 4 Futures
Ejercicio 4: Distribución de tareas dinámicas

1. Implementa una función que calcule la raíz cuadrada de una lista de 100,000 números enteros generados aleatoriamente.
2. Divide la lista en 10 partes iguales y usa Dask Futures para calcular la raíz cuadrada de cada parte en paralelo.
3. Recolecta los resultados y calcula:
    - El promedio de todos los números procesados.
    - El tiempo total de ejecución (incluyendo envío y recolección de tareas).
4. Observa como se distribuye la carga en el cliente.

*Nota*: en los ejercicios ya vimos como determinar si ya se cumplío una tarea.

In [ ]:

# Paso 1: Configurar el cliente de Dask
client = Client()  # Esto inicia un clúster local automáticamente

# Paso 2: Generar una lista de 100,000 números enteros aleatorios
tamaño_lista = 100_000
numeros = np.random.randint(1, 10_000, size=tamaño_lista)

# Paso 3: Definir la función para calcular raíces cuadradas
def calcular_raiz_cuadrada(lista):
    return np.sqrt(lista)

# Dividir la lista en 10 partes iguales
partes = np.array_split(numeros, 10)

# Paso 4: Usar Futures para distribuir tareas
start_time = time.time()
futures = [client.submit(calcular_raiz_cuadrada, parte) for parte in partes]

# Recolectar los resultados
resultados = client.gather(futures)

# Paso 5: Calcular el promedio de todos los números procesados
todos_los_resultados = np.concatenate(resultados)
promedio = todos_los_resultados.mean()
tiempo_total = time.time() - start_time


print(f"\nPromedio de las raíces cuadradas: {promedio:.2f}")
print(f"Tiempo total de ejecución: {tiempo_total:.2f} segundos")
print("\nDashboard disponible en:")
print(client.dashboard_link)


c:\Users\Oscar\Documents\5to Semestre\Fuentes de datos\Exposiciones\Tarea_Dask\.venv\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56547 instead
  warnings.warn(



Promedio de las raíces cuadradas: 66.71
Tiempo total de ejecución: 0.04 segundos

Dashboard disponible en:
http://127.0.0.1:56547/status
